In [ ]:
#SE DEVI PRENDERE ALTRI COMMENTI RICORDATI CHE HAI DIVISO I FILE DEI VIDEO

In [1]:
import pandas as pd

import requests 

import os

import json

import googleapiclient.discovery
import googleapiclient.errors

import numpy as np

In [2]:
chiave_yt = json.load(open('key.json', 'r'))['youtube']['key'][0]
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=chiave_yt)

In [3]:
#insert initial quota value
quota = 0

In [4]:
df_videos = pd.read_csv('data/get_youtuber_videos_output/youtuber_videos.csv')

In [5]:
def get_infos(response):
    info_comment_list=[]
    video_id=next_page_token = thread_comment_id = top_level_comment_id = author_display_name = author_channel_id = author_channel_id = text_original=None
    try:
        next_page_token = response['nextPageToken']
    except KeyError as e:
        #print(response)
        next_page_token = None
        print('Page token non esistente')
    items = response['items']
    for item in items:
        thread_comment_id = item['id']
        try:
            video_id = item['snippet']['videoId']
            top_level_comment = item['snippet']['topLevelComment']
            top_level_comment_id = top_level_comment['id']
            top_level_comment_snippet = top_level_comment['snippet']
            author_display_name = top_level_comment_snippet['authorDisplayName']
            text_original = top_level_comment_snippet['textOriginal']
            author_channel_id = top_level_comment_snippet['authorChannelId']['value']
        except KeyError as e:
            print(e.args[0])
            print('Problem with commentThread {}'.format(thread_comment_id))
        info_comment_list.append([video_id,thread_comment_id,top_level_comment_id,author_display_name,author_channel_id,text_original])
    return next_page_token, info_comment_list

In [25]:
quota

9903

In [6]:
#############################################################################

In [7]:
#round 1
#la prossima volta togli e decommenta quello sotto

In [8]:
last_csv = int(np.load('data/get_comment_threads_output/last_csv.npy'))
next_page_token = str(np.load('data/get_comment_threads_output/next_page_token.npy'))
last_video_idx = int(np.load('data/get_comment_threads_output/video_to_start.npy'))
videos_not_finished = (np.load('data/get_comment_threads_output/videos_not_finished.npy')).tolist()
videos_disabled_comments = (np.load('data/get_comment_threads_output/videos_disabled_comments.npy')).tolist()
print("Starting with video #{}, next_page_token: {}".format(last_video_idx,next_page_token))
print("Writing on {} csv".format(last_csv+1))

Starting with video #1733, next_page_token: QURTSl9pMGVXd2RTZVc4TTFHRUZaYWowYUkxc21qYTRHdGpkWUd3dlFpazFnaTl3NTc1RVE5dGJ6RVNQTkJEaVdncXExc2w2U1VUbUJvOUo3enJuR25rY0lSOUVJWmNmY0I0QjZfaXMwN0NLMjYtdDd6RzRHMWlpZjVrNDFEU1lyalU=
Writing on 4 csv


In [9]:
df_videos_0 = df_videos[last_video_idx:]

In [10]:
#per i video con più di 10000 commenti, salvo l'indice del video con il next page token

In [11]:
comments = []
comments_count = 0

In [12]:
df_videos_0

,channel_id,video_id
1733,UC0f_USnWT_ll5SHhmr9ZKpA,96Hl-mEcnWQ
1734,UC0f_USnWT_ll5SHhmr9ZKpA,MxrR3TCBs6M
1735,UC0f_USnWT_ll5SHhmr9ZKpA,DsHwC_KPWWw
1736,UC0f_USnWT_ll5SHhmr9ZKpA,t-hJgcFwVzw
1737,UC0f_USnWT_ll5SHhmr9ZKpA,wznsr1iCd1U
...,...,...
2588,UCq7EY7H2XF6TV7Z5mLv-aNg,bm-3Qrr7EUA
2589,UCq7EY7H2XF6TV7Z5mLv-aNg,hkxRjDS_Cjw
2590,UCq7EY7H2XF6TV7Z5mLv-aNg,8kElrRAa1ws
2591,UCq7EY7H2XF6TV7Z5mLv-aNg,7x6tX5T7LOs


In [13]:
for video in df_videos_0['video_id'].values:
    print('________________________________________________________')
    print('{} video: {}'.format(last_video_idx, video))
    video_comments_count = 0
    while next_page_token != None and video_comments_count<10000:
        print('Reached quotas: {}'.format(quota))
        if quota > 9900:
            print('Quota exceeded')
            break
        else:
            request = youtube.commentThreads().list(
                part="snippet",
                maxResults=100,
                pageToken=next_page_token,
                videoId=video
            )
            try:
                response = request.execute()
                quota += 3
                next_page_token, results = get_infos(response)
                video_comments_count+=len(results)
                comments_count+=video_comments_count
                comments.append(results)
            except:
                videos_disabled_comments.append(video)
                next_page_token = None
                print('Video has disabled comments')
    if quota > 9900:
        break
    if next_page_token == None or video_comments_count>=10000:
        if next_page_token != None:
            print('Reached max comment count')
            videos_not_finished.append([video, next_page_token])
        last_video_idx += 1
        next_page_token = ' '
        print('Total comments for {}: {}'.format(video, video_comments_count))

print("Done with {} comments, video to start: {}\n with next_page_token: {}".format(comments_count,last_video_idx, next_page_token))

________________________________________________________
1733 video: 96Hl-mEcnWQ
Reached quotas: 0
Page token non esistente
Total comments for 96Hl-mEcnWQ: 30
________________________________________________________
1734 video: MxrR3TCBs6M
Reached quotas: 3
Reached quotas: 6
Reached quotas: 9
Reached quotas: 12
Reached quotas: 15
Reached quotas: 18
Reached quotas: 21
Reached quotas: 24
Reached quotas: 27
Reached quotas: 30
Reached quotas: 33
Page token non esistente
Total comments for MxrR3TCBs6M: 1023
________________________________________________________
1735 video: DsHwC_KPWWw
Reached quotas: 36
Reached quotas: 39
Page token non esistente
Total comments for DsHwC_KPWWw: 199
________________________________________________________
1736 video: t-hJgcFwVzw
Reached quotas: 42
Reached quotas: 45
Reached quotas: 48
Reached quotas: 51
Page token non esistente
Total comments for t-hJgcFwVzw: 321
________________________________________________________
1737 video: wznsr1iCd1U
Reached quota

Page token non esistente
Total comments for -K5TmI3fh90: 94
________________________________________________________
1778 video: Jo3fBta4nJw
Reached quotas: 291
Page token non esistente
Total comments for Jo3fBta4nJw: 77
________________________________________________________
1779 video: T49_yKLey5U
Reached quotas: 294
Page token non esistente
Total comments for T49_yKLey5U: 79
________________________________________________________
1780 video: 8-Ky4nOEQwY
Reached quotas: 297
Page token non esistente
Total comments for 8-Ky4nOEQwY: 82
________________________________________________________
1781 video: Yz2lM8nvvE0
Reached quotas: 300
Page token non esistente
Total comments for Yz2lM8nvvE0: 84
________________________________________________________
1782 video: a9CFTPLnDng
Reached quotas: 303
Page token non esistente
Total comments for a9CFTPLnDng: 66
________________________________________________________
1783 video: 6h0we2nP6B8
Reached quotas: 306
Page token non esistente
Total com

Reached quotas: 714
Reached quotas: 717
Reached quotas: 720
Page token non esistente
Total comments for f3HIIBUIx70: 375
________________________________________________________
1817 video: -WxJ5K90yEE
Reached quotas: 723
Reached quotas: 726
Reached quotas: 729
Page token non esistente
Total comments for -WxJ5K90yEE: 239
________________________________________________________
1818 video: cdlLnxmVx6U
Reached quotas: 732
Reached quotas: 735
Page token non esistente
Total comments for cdlLnxmVx6U: 147
________________________________________________________
1819 video: DQd7gyStwhg
Reached quotas: 738
Reached quotas: 741
Reached quotas: 744
Reached quotas: 747
Page token non esistente
Total comments for DQd7gyStwhg: 383
________________________________________________________
1820 video: s57jB-VVUWY
Reached quotas: 750
Reached quotas: 753
Reached quotas: 756
Page token non esistente
Total comments for s57jB-VVUWY: 262
________________________________________________________
1821 video: Fh

Reached quotas: 1302
Reached quotas: 1305
Reached quotas: 1308
Page token non esistente
Total comments for tnUs1a8RwSM: 354
________________________________________________________
1847 video: Eu7Ka2B7VrM
Reached quotas: 1311
Reached quotas: 1314
Reached quotas: 1317
Reached quotas: 1320
Reached quotas: 1323
Reached quotas: 1326
Reached quotas: 1329
Reached quotas: 1332
Reached quotas: 1335
Reached quotas: 1338
Page token non esistente
Total comments for Eu7Ka2B7VrM: 996
________________________________________________________
1848 video: tGD0i5y-U7U
Reached quotas: 1341
Reached quotas: 1344
Reached quotas: 1347
Reached quotas: 1350
Reached quotas: 1353
Reached quotas: 1356
Reached quotas: 1359
Reached quotas: 1362
Reached quotas: 1365
Reached quotas: 1368
Reached quotas: 1371
Reached quotas: 1374
Reached quotas: 1377
Reached quotas: 1380
Reached quotas: 1383
Reached quotas: 1386
Reached quotas: 1389
Page token non esistente
Total comments for tGD0i5y-U7U: 1685
________________________

Reached quotas: 1605
Page token non esistente
Total comments for dkx4Ie9Ertw: 309
________________________________________________________
1890 video: 5CuGmzgd4BM
Reached quotas: 1608
Reached quotas: 1611
Reached quotas: 1614
Reached quotas: 1617
Reached quotas: 1620
Reached quotas: 1623
Page token non esistente
Total comments for 5CuGmzgd4BM: 559
________________________________________________________
1891 video: Zpc7uyvose8
Reached quotas: 1626
Reached quotas: 1629
Reached quotas: 1632
Reached quotas: 1635
Reached quotas: 1638
Reached quotas: 1641
Page token non esistente
Total comments for Zpc7uyvose8: 562
________________________________________________________
1892 video: AzJH8M7zHjs
Reached quotas: 1644
Reached quotas: 1647
authorChannelId
Problem with commentThread Ugyio_777sQQCuvbyJV4AaABAg
Reached quotas: 1650
Reached quotas: 1653
Reached quotas: 1656
Reached quotas: 1659
Reached quotas: 1662
Reached quotas: 1665
Reached quotas: 1668
Reached quotas: 1671
Reached quotas: 1674


...

Reached quotas: 6015
Page token non esistente
Total comments for WTsRVxH-G8k: 160
________________________________________________________
2310 video: XGJvGOLW8jA
Reached quotas: 6018
Reached quotas: 6021
Page token non esistente
Total comments for XGJvGOLW8jA: 174
________________________________________________________
2311 video: rkQMKoHXASg
Reached quotas: 6024
Reached quotas: 6027
Reached quotas: 6030
Page token non esistente
Total comments for rkQMKoHXASg: 281
________________________________________________________
2312 video: DBEFjIKxr7E
Reached quotas: 6033
Reached quotas: 6036
Page token non esistente
Total comments for DBEFjIKxr7E: 142
________________________________________________________
2313 video: RYkBKwk8jmQ
Reached quotas: 6039
Reached quotas: 6042
Reached quotas: 6045
Reached quotas: 6048
Page token non esistente
Total comments for RYkBKwk8jmQ: 392
________________________________________________________
2314 video: Ey2i4cqVKCY
Reached quotas: 6051
Reached quotas: 6

Reached quotas: 6393
Page token non esistente
Total comments for 9xsNKthpKvQ: 249
________________________________________________________
2349 video: CsdZQcyyXRw
Reached quotas: 6396
Reached quotas: 6399
Reached quotas: 6402
Page token non esistente
Total comments for CsdZQcyyXRw: 281
________________________________________________________
2350 video: BYFYqPkxkEY
Reached quotas: 6405
Reached quotas: 6408
Page token non esistente
Total comments for BYFYqPkxkEY: 153
________________________________________________________
2351 video: QT2Qwev0Zt8
Reached quotas: 6411
Reached quotas: 6414
Page token non esistente
Total comments for QT2Qwev0Zt8: 154
________________________________________________________
2352 video: g-PxZbEbojE
Reached quotas: 6417
Reached quotas: 6420
Reached quotas: 6423
Page token non esistente
Total comments for g-PxZbEbojE: 206
________________________________________________________
2353 video: _i9sWXNTTuo
Reached quotas: 6426
Reached quotas: 6429
Reached quotas: 6

Reached quotas: 7239
Reached quotas: 7242
Page token non esistente
Total comments for qznzzkEf0yM: 1644
________________________________________________________
2365 video: I8eDA5Mp4DI
Reached quotas: 7245
Reached quotas: 7248
Reached quotas: 7251
Reached quotas: 7254
Reached quotas: 7257
Reached quotas: 7260
Reached quotas: 7263
Reached quotas: 7266
Reached quotas: 7269
Reached quotas: 7272
Reached quotas: 7275
Reached quotas: 7278
Reached quotas: 7281
Reached quotas: 7284
Reached quotas: 7287
Reached quotas: 7290
Reached quotas: 7293
Reached quotas: 7296
Reached quotas: 7299
Reached quotas: 7302
Reached quotas: 7305
Reached quotas: 7308
Reached quotas: 7311
Reached quotas: 7314
Reached quotas: 7317
Page token non esistente
Total comments for I8eDA5Mp4DI: 2415
________________________________________________________
2366 video: kcaAMh8Z1fw
Reached quotas: 7320
Reached quotas: 7323
Reached quotas: 7326
Reached quotas: 7329
Reached quotas: 7332
Reached quotas: 7335
Reached quotas: 7338


Reached quotas: 8187
Reached quotas: 8190
Reached quotas: 8193
Reached quotas: 8196
Reached quotas: 8199
Reached quotas: 8202
Reached quotas: 8205
Reached quotas: 8208
Reached quotas: 8211
Reached quotas: 8214
Reached quotas: 8217
Reached quotas: 8220
Reached quotas: 8223
Reached quotas: 8226
Reached quotas: 8229
Reached quotas: 8232
Reached quotas: 8235
Reached quotas: 8238
Page token non esistente
Total comments for ibU0KkOtpl0: 2879
________________________________________________________
2376 video: h4pdS7RaDxY
Reached quotas: 8241
Reached quotas: 8244
Reached quotas: 8247
Reached quotas: 8250
Reached quotas: 8253
Reached quotas: 8256
Reached quotas: 8259
Reached quotas: 8262
Reached quotas: 8265
Reached quotas: 8268
Reached quotas: 8271
Reached quotas: 8274
Reached quotas: 8277
Reached quotas: 8280
Reached quotas: 8283
Reached quotas: 8286
Reached quotas: 8289
Reached quotas: 8292
Reached quotas: 8295
Reached quotas: 8298
Reached quotas: 8301
Reached quotas: 8304
Reached quotas: 8

Reached quotas: 9132
Reached quotas: 9135
Reached quotas: 9138
Page token non esistente
Total comments for DGTGKyNoeqs: 1782
________________________________________________________
2387 video: YArEp6FavJY
Reached quotas: 9141
Reached quotas: 9144
Reached quotas: 9147
Reached quotas: 9150
Reached quotas: 9153
Reached quotas: 9156
Reached quotas: 9159
Reached quotas: 9162
Reached quotas: 9165
Reached quotas: 9168
Reached quotas: 9171
Reached quotas: 9174
Reached quotas: 9177
Reached quotas: 9180
Reached quotas: 9183
Reached quotas: 9186
Reached quotas: 9189
Reached quotas: 9192
Reached quotas: 9195
Reached quotas: 9198
Reached quotas: 9201
Reached quotas: 9204
Reached quotas: 9207
Reached quotas: 9210
Reached quotas: 9213
Reached quotas: 9216
Reached quotas: 9219
Reached quotas: 9222
Reached quotas: 9225
Reached quotas: 9228
Reached quotas: 9231
Reached quotas: 9234
Reached quotas: 9237
Reached quotas: 9240
Page token non esistente
Total comments for YArEp6FavJY: 3364
__________________

In [14]:
np.save('data/get_comment_threads_output/next_page_token.npy',next_page_token)
np.save('data/get_comment_threads_output/video_to_start.npy',last_video_idx)
np.save('data/get_comment_threads_output/videos_not_finished.npy',videos_not_finished)
np.save('data/get_comment_threads_output/videos_disabled_comments.npy',videos_disabled_comments)

In [15]:
print(next_page_token, last_video_idx)

QURTSl9pMGRlWWU5S2k3SzZsR1VoaUdad2FmejFMYl8xMC1sa3dWRk5uRnZJUlpPODdGcW5HQ3U2a1h2MllRVFZIbzFJM3oyWVFHcE5mZ3NkY0tqWXlRMTFTb1V2RHhzREtWbkRXb25IWEpSWUhRWU52MmF0NkM0eDMtUmhoNDNJb200bGxJMXgtN3RDdkJoSGRJdFZ3RmxmS3BMOUxUbWlRR0NnOV9faEtUVQ== 2399


In [16]:
format_ok_comments = []
for outer_comment in comments:
    for comment in outer_comment:
        format_ok_comments.append(comment)

In [17]:
format_ok_comments

[['96Hl-mEcnWQ',
  'UgzjEZt3-vteXnlPqvx4AaABAg',
  'UgzjEZt3-vteXnlPqvx4AaABAg',
  'naturalbeauty511',
  'UCoPogfpPUcRtqiNxlhvvhTg',
  'The Nars sheer glow and natural radiant shades run completely different. I wear a totally different shade. in sheer glow'],
 ['96Hl-mEcnWQ',
  'Ugwzf1WOR47qTjy0lQd4AaABAg',
  'Ugwzf1WOR47qTjy0lQd4AaABAg',
  'Beauty by Crisha',
  'UCrziqYRc3uyaepEq9Hd2KtA',
  'Yas T cute'],
 ['96Hl-mEcnWQ',
  'Ugzt8uo4GGxpcqVXVll4AaABAg',
  'Ugzt8uo4GGxpcqVXVll4AaABAg',
  'Boubou',
  'UCC_Sg8KvPcFXpQctQsoKEkg',
  'Pretty look but i feel like its too much skin products especially for summer'],
 ['96Hl-mEcnWQ',
  'UgyFfp28El3JfV22HEh4AaABAg',
  'UgyFfp28El3JfV22HEh4AaABAg',
  'Maine Wonders',
  'UCWVb6y8pQWg5ssXZ4wDKsNQ',
  'MY LOVER!!! A FAV! I wish your thumbnail was a snap from your intro! because sis, that hat?? that all white look yes!!!\nWho is with me?'],
 ['96Hl-mEcnWQ',
  'UgxlZ-89yxAOtgXsmcp4AaABAg',
  'UgxlZ-89yxAOtgXsmcp4AaABAg',
  'Kara Cee',
  'UCodGcL4TmO7Q

In [18]:
df_comment_thread = pd.DataFrame(format_ok_comments,columns=['video_id','comment_thread_id','top_level_comment_id','author_display_name','author_channel_id','text_original'])

In [19]:
df_comment_thread

,video_id,comment_thread_id,top_level_comment_id,author_display_name,author_channel_id,text_original
0,96Hl-mEcnWQ,UgzjEZt3-vteXnlPqvx4AaABAg,UgzjEZt3-vteXnlPqvx4AaABAg,naturalbeauty511,UCoPogfpPUcRtqiNxlhvvhTg,The Nars sheer glow and natural radiant shades...
1,96Hl-mEcnWQ,Ugwzf1WOR47qTjy0lQd4AaABAg,Ugwzf1WOR47qTjy0lQd4AaABAg,Beauty by Crisha,UCrziqYRc3uyaepEq9Hd2KtA,Yas T cute
2,96Hl-mEcnWQ,Ugzt8uo4GGxpcqVXVll4AaABAg,Ugzt8uo4GGxpcqVXVll4AaABAg,Boubou,UCC_Sg8KvPcFXpQctQsoKEkg,Pretty look but i feel like its too much skin ...
3,96Hl-mEcnWQ,UgyFfp28El3JfV22HEh4AaABAg,UgyFfp28El3JfV22HEh4AaABAg,Maine Wonders,UCWVb6y8pQWg5ssXZ4wDKsNQ,MY LOVER!!! A FAV! I wish your thumbnail was a...
4,96Hl-mEcnWQ,UgxlZ-89yxAOtgXsmcp4AaABAg,UgxlZ-89yxAOtgXsmcp4AaABAg,Kara Cee,UCodGcL4TmO7Q8a9-ah_9p-Q,"If I take a sip every time she says “glowy”, I..."
...,...,...,...,...,...,...
296607,lSlKqgyE2Zo,UgztWbHI323ee5B85_54AaABAg,UgztWbHI323ee5B85_54AaABAg,Kerri Simonsen,UC1yNObi3scDTvrFWuOueYLw,"I have DEFINITELY had doctors and still do, wh..."
296608,lSlKqgyE2Zo,UgwGjO7-dYohYq_FPQ14AaABAg,UgwGjO7-dYohYq_FPQ14AaABAg,Mom Of 4,UCqJOp6R1VtJWBxmOLs5tB9Q,Don’t apologize for being upset about demoneti...
296609,lSlKqgyE2Zo,Ugw-xJgix05O5FppBKd4AaABAg,Ugw-xJgix05O5FppBKd4AaABAg,Sunni The Bunny,UCkW2PVkmFfxL0krrEiTUsBw,Okay. Seriously. There's NOTHING nice to say a...
296610,lSlKqgyE2Zo,UgxBnI_IBZRq9WT2p8h4AaABAg,UgxBnI_IBZRq9WT2p8h4AaABAg,DreamscapedJazz,UCs-ZFVAgsztkk24wM8GKNmA,"That doctor is a fucking liar, PCOS, which I h..."


In [20]:
last_csv

3

In [21]:
last_csv+=1
last_csv

4

In [22]:
df_comment_thread.to_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv), index = False)

In [23]:
np.save('data/get_comment_threads_output/last_csv.npy',last_csv)

In [24]:
df = pd.read_csv('data/get_comment_threads_output/df_comment_thread_{}.csv'.format(last_csv))
df

,video_id,comment_thread_id,top_level_comment_id,author_display_name,author_channel_id,text_original
0,96Hl-mEcnWQ,UgzjEZt3-vteXnlPqvx4AaABAg,UgzjEZt3-vteXnlPqvx4AaABAg,naturalbeauty511,UCoPogfpPUcRtqiNxlhvvhTg,The Nars sheer glow and natural radiant shades...
1,96Hl-mEcnWQ,Ugwzf1WOR47qTjy0lQd4AaABAg,Ugwzf1WOR47qTjy0lQd4AaABAg,Beauty by Crisha,UCrziqYRc3uyaepEq9Hd2KtA,Yas T cute
2,96Hl-mEcnWQ,Ugzt8uo4GGxpcqVXVll4AaABAg,Ugzt8uo4GGxpcqVXVll4AaABAg,Boubou,UCC_Sg8KvPcFXpQctQsoKEkg,Pretty look but i feel like its too much skin ...
3,96Hl-mEcnWQ,UgyFfp28El3JfV22HEh4AaABAg,UgyFfp28El3JfV22HEh4AaABAg,Maine Wonders,UCWVb6y8pQWg5ssXZ4wDKsNQ,MY LOVER!!! A FAV! I wish your thumbnail was a...
4,96Hl-mEcnWQ,UgxlZ-89yxAOtgXsmcp4AaABAg,UgxlZ-89yxAOtgXsmcp4AaABAg,Kara Cee,UCodGcL4TmO7Q8a9-ah_9p-Q,"If I take a sip every time she says “glowy”, I..."
...,...,...,...,...,...,...
296607,lSlKqgyE2Zo,UgztWbHI323ee5B85_54AaABAg,UgztWbHI323ee5B85_54AaABAg,Kerri Simonsen,UC1yNObi3scDTvrFWuOueYLw,"I have DEFINITELY had doctors and still do, wh..."
296608,lSlKqgyE2Zo,UgwGjO7-dYohYq_FPQ14AaABAg,UgwGjO7-dYohYq_FPQ14AaABAg,Mom Of 4,UCqJOp6R1VtJWBxmOLs5tB9Q,Don’t apologize for being upset about demoneti...
296609,lSlKqgyE2Zo,Ugw-xJgix05O5FppBKd4AaABAg,Ugw-xJgix05O5FppBKd4AaABAg,Sunni The Bunny,UCkW2PVkmFfxL0krrEiTUsBw,Okay. Seriously. There's NOTHING nice to say a...
296610,lSlKqgyE2Zo,UgxBnI_IBZRq9WT2p8h4AaABAg,UgxBnI_IBZRq9WT2p8h4AaABAg,DreamscapedJazz,UCs-ZFVAgsztkk24wM8GKNmA,"That doctor is a fucking liar, PCOS, which I h..."
